In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
179,AFG,Asia,Afghanistan,2020-07-07,33384.0,433.0,920.0,56.0,857.576,11.123,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
300,ALB,Europe,Albania,2020-07-07,2964.0,71.0,79.0,3.0,1029.953,24.672,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
485,DZA,Africa,Algeria,2020-07-07,16404.0,463.0,959.0,7.0,374.085,10.558,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-06,France,167204.0,0.0,29897.0,0.0,0.0,0.0
0,2020-07-06,China,84922.0,0.0,4651.0,0.0,0.0,0.0
0,2020-07-06,Italy,241974.0,0.0,34906.0,0.0,0.0,0.0
0,2020-07-06,Spain,250650.0,0.0,28400.0,0.0,0.0,0.0
0,2020-07-06,United States,2932197.0,0.0,130446.0,0.0,0.0,0.0
0,2020-07-06,World,11646780.0,0.0,539510.0,0.0,0.0,0.0
0,2020-07-06,United Kingdom,286431.0,0.0,44345.0,0.0,0.0,0.0
0,2020-07-06,Germany,196891.0,0.0,9028.0,0.0,0.0,0.0
0,2020-07-06,Iran,243596.0,0.0,11708.0,0.0,0.0,0.0
0,2020-07-06,Turkey,206845.0,0.0,5244.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-07,France,167204.0,168335.0,29897.0,29920.0,-1131.0,-23.0
0,2020-07-07,China,84922.0,84896.0,4651.0,4641.0,26.0,10.0
0,2020-07-07,Italy,241974.0,241819.0,34906.0,34869.0,155.0,37.0
0,2020-07-06,Spain,250650.0,251789.0,28400.0,28388.0,-1139.0,12.0
0,2020-07-07,United States,2932197.0,2938625.0,130446.0,130306.0,-6428.0,140.0
0,2020-07-07,World,11646780.0,11586205.0,539510.0,537701.0,60575.0,1809.0
0,2020-07-07,United Kingdom,286431.0,285768.0,44345.0,44236.0,663.0,109.0
0,2020-07-07,Germany,196891.0,196944.0,9028.0,9024.0,-53.0,4.0
0,2020-07-07,Iran,243596.0,243051.0,11708.0,11731.0,545.0,-23.0
0,2020-07-07,Turkey,206845.0,206844.0,5244.0,5241.0,1.0,3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")